In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
import urllib
from bs4 import BeautifulSoup as bs
import json
import datetime
from tqdm.notebook import tqdm
import pymysql
import traceback
import psycopg2
from psycopg2.extras import RealDictCursor
from appstoreconnect import Api
import pymssql
from dateutil.relativedelta import relativedelta
import pickle as pk

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

## mall.csv

In [6]:
cols = ["주문번호", "아이디", "상품ID", "상품명", "수량", "상품금액", "주문일자", "모바일주문여부"]

with open("mall_password.pkl", "rb") as f:
     password = pk.load(f)
conn = pymssql.connect(server="125.60.68.233", database="eiparkclub", user="myhomie", password=password, charset="utf8")
cur = conn.cursor()

sql = f"SELECT 주문번호, 임직원아이디, _상품ID, 상품명, 수량, 상품금액, 주문일자, _모바일주문여부 FROM vwAdjustDetail;"
# sql = f"SELECT 임직원아이디 FROM vwAdjustDetail;"
cur.execute(sql)

data = pd.DataFrame(cur.fetchall(), columns=cols)
# data = pd.DataFrame(cur.fetchall())
data = data[data["아이디"].str.contains("MBR")]
data = data.reset_index(drop=True)

In [5]:
# data.to_csv("mall.csv", encoding="euc-kr", index=False)
# data.head()

# 다운로드(iOS)

In [118]:
with open("D:/AuthKey_BB99VD4S7X.p8", "r") as f:
    key_file = f.read()
api = Api(key_id="BB99VD4S7X", key_file=key_file, issuer_id="252fd97c-ea6e-45b9-a795-693f38f8dbaf")
api.download_sales_and_trends_reports(filters={"frequency": "DAILY", "reportSubType":"DETAILED", "reportType":"SALES", "vendorNumber": "88755846"})
# , save_to="report.csv")

APIError: Invalid vendor number specified. Try again.

## 구글 애널리틱스

### pageviews.csv

In [3]:
# hdcdvp.homie@gmail.com
# ghalghkdlxld!
# yesterday = (datetime.today() - timedelta(days=1))
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2021, 5, 31)
token = "ya29.a0AfH6SMB0ZXx4iXFHI4akXbVmboQi7xzSglctKuX-nZnBHMmAAkEpADxBqGFv6xy8WSNDQzCnplepFPCjwsvhTFR9KG7jhV_s1ePkf0sGDqbInR-L8VtkBaYyuKk214zY3vRnYB8C-van-4zwmOXInCeI84a-E1U"

In [3]:
views = pd.DataFrame()
for i in tqdm(range(100)):
    try:
        start = end - datetime.timedelta(days=10*(i+1))
        end = end - datetime.timedelta(days=10*i)
        url = f"https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A206525354&start-date={start.strftime('%Y-%m-%d')}&end-date={end.strftime('%Y-%m-%d')}&metrics=ga%3Ausers%2Cga%3Apageviews&dimensions=ga%3ApageTitle%2Cga%3Adate&sort=-ga%3Adate&access_token={token}"
        req = urllib.request.urlopen(url).read().decode("utf8")
        js = json.loads(req)
        
        df = pd.DataFrame(js["rows"], columns=pd.DataFrame(js["columnHeaders"])["name"].str.split(":").str[1].tolist())
        views = pd.concat([views, df])
    except:
        break

In [7]:
# views.tail()
# views.to_excel(f"pageviews_{end}.xlsx")

### MAU

In [4]:
url = f"https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A206525354&start-date={start.strftime('%Y-%m-%d')}&end-date={end.strftime('%Y-%m-%d')}&metrics=ga%3Ausers%2Cga%3Apageviews&dimensions=ga%3AyearMonth&access_token={token}"
req = urllib.request.urlopen(url).read().decode("utf8")
js = json.loads(req)

mau = pd.DataFrame(js["rows"], columns=pd.DataFrame(js["columnHeaders"])["name"].str.split(":").str[1].tolist())

In [8]:
# mau.tail()
# mau.to_csv(f"mau_2021-05-31.csv", encoding="euc-kr", index=False)

## 개발사 데이터

## sys_code

In [9]:
with open("signup_password.pkl", "wb") as f:
	pk.dump("dtRW2aj14KYl7aQl8oZd", f)

In [16]:
with open("signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = f"SELECT * FROM hdc_sys.sys_code;"
cur.execute(query)
result = cur.fetchall()

sys_code = pd.DataFrame(result)

In [18]:
# sys_code.head()

## apt_mst.csv

In [8]:
with open("signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = f"SELECT * FROM hdc_apt.apt_mst;"
cur.execute(query)
result = cur.fetchall()

apt_mst = pd.DataFrame(result)

In [19]:
# apt_mst.to_csv("apt_mst_2021-05-31.csv", encoding="euc-kr",  index=False)
# apt_mst.head()

## signup.csv

In [27]:
with open("signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = f"SELECT hdc_mbr.mbr_mst.mbr_id, hdc_mbr.mbr_mst.mbr_sys_ty, hdc_mbr.mbr_mst.mbr_sex, hdc_mbr.mbr_mst.mbr_birth, hdc_mbr.mbr_mst.regist_dt, hdc_mbr.mbr_mst.mbr_delete_date, hdc_mbr.mbr_mst.mbr_st,\
hdc_apt.apt_unit_mbr.mbr_id,\
hdc_apt.apt_unit_info.dng_no, hdc_apt.apt_unit_info.ho_no,\
hdc_apt.apt_mst.apt_name \
FROM hdc_mbr.mbr_mst \
LEFT OUTER JOIN hdc_apt.apt_unit_mbr \
LEFT OUTER JOIN hdc_apt.apt_unit_info \
LEFT OUTER JOIN hdc_apt.apt_mst \
ON hdc_apt.apt_unit_info.apt_id = hdc_apt.apt_mst.apt_id \
ON hdc_apt.apt_unit_mbr.unit_id = hdc_apt.apt_unit_info.unit_id \
ON hdc_mbr.mbr_mst.mbr_id = hdc_apt.apt_unit_mbr.mbr_id;"
cur.execute(query)
result = cur.fetchall()
signup = pd.DataFrame(result)

In [23]:
signup = signup[signup["apt_name"].notna()]
signup["mbr_sys_ty"] = signup["mbr_sys_ty"].map({"C000004001":"입주예정자", "C000004001":"입주자(대표)", "C000004001":"입주자(세대원)"})
signup["mbr_sex"] = signup["mbr_sex"].map({"0":"남자", "1":"여자"})
signup["mbr_st"] = signup["mbr_st"].map({"C000002001":"정상", "C000002005":"탈퇴", "C000002007":"퇴거", "C000002008":"휴면상태"})
signup = signup.rename({"mbr_id":"아이디", "apt_name":"아파트 명", "dng_no":"동", "ho_no":"호", "mbr_sys_ty":"회원유형", "mbr_st":"회원상태", "mbr_sex":"성별", "mbr_birth":"생년월일", "regist_dt":"가입일", "mbr_delete_date":"탈퇴일"}, axis=1)
signup["성별/생년월일"] = signup["성별"] + "/" + signup["생년월일"]
signup = signup[["아이디", "아파트 명", "동", "호", "회원유형", "회원상태", "성별/생년월일", "가입일", "탈퇴일"]]

In [13]:
# signup.to_csv("D:\디지털혁신팀\태블로\마이호미\마이호미_FINAL\signup_2021-05-31.csv", encoding="euc-kr")
# signup.head()